Changing approaches

Direct approaches to create bschema have too many compromises and aren't working generically. 

Should change approaches to use one hop graph queries and LLMs to do new class names, then can just create a class graph. 

This also has more interpretability than the bschema, where you still had to inspect the graph to learn what something is, instead of just looking at a human readable label. 

The amount of iterations of naming things based on their one hop graphs should also work for different schema or types of graphs so it is somewhat manual but also can be generalized. 